# Implementation Recovering OGM

In [1]:
using JuMP
using Ipopt

# (1) Problem size
N = 5  # you can change this for testing

# (2) Build the model with Ipopt
model = Model(Ipopt.Optimizer)

set_optimizer_attribute(model, "print_level",             0)
set_optimizer_attribute(model, "print_timing_statistics", "no")
set_optimizer_attribute(model, "print_user_options",      "no")
set_optimizer_attribute(model, "tol", 1e-8)
set_optimizer_attribute(model, "max_iter", 5000)

# (3) Decision variables:
#     a[i] ≥ 0 for i = 0,…,N
#     λ[i,j] ≥ 0 for i,j = 0,…,N
#    JuMP supports zero‐based indexing directly:
@variable(model, a[0:N])
@variable(model, λ[0:N, 0:N] >= 0)

# (4) Constraints
for i in 0:N
    # zero diagonal
    @constraint(model, λ[i,i] == 0)


    for j in 0:i
        @constraint(model, λ[i,j] == 0)
    end

    # flow‐balance constraints
    if i < N
        @constraint(model,
            sum(λ[i,j] - λ[j,i] for j in 0:N) == a[i]
        )
    else
        # for i = N, RHS is -sum_j a_j
        @constraint(model,
            sum(λ[N,j] - λ[j,N] for j in 0:N) == -1*sum(a[j] for j in 0:(N-1))
        )
    end

    # nonlinear “capacity” constraint:
    #   ∑_j (–λ_ij – λ_ji) + a_i + a_i^2 ≤ 0
    @NLconstraint(model,
        sum(-λ[i,j] - λ[j,i] for j in 0:N) - a[i] + a[i]^2 <= 0
    )
end

# (5) Objective: maximize ∑_i a_i
@objective(model, Max, sum(a[i] for i in 0:N))

# (6) Solve
optimize!(model)

# (7) Report results
println("Status: ", termination_status(model))
println("Objective (∑ a_i): ", objective_value(model))
println("a = ", value.(a))
println("λ = ")
for i in 0:N, j in 0:N
    val = value(λ[i,j])
    if abs(val) <= 1e-6
        val=0
    end
    print(val, j == N ? "\n" : " ")
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Status: LOCALLY_SOLVED
Objective (∑ a_i): 26.898876584900357
a = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:5
And data, a 6-element Vector{Float64}:
 1.999999944183546
 3.236067923466534
 4.387054115754427
 5.499582624488791
 6.589759300459151
 5.186412676547909
λ = 
0 1.9999999268726358 0 0 0 0
0 0 5.2360678391195306 0 0 0
0 0 0 9.623121943812368 0 0
0 0 0 0 15.122704562760223 0
0 0 0 0 0 21.712463876632764
0 0 0 0 0 0


In [ ]:
compute_H(5)

5×5 Matrix{Float64}:
 1.61803    0.0       0.0       0.0       0.0
 0.174133   2.01939   0.0       0.0       0.0
 0.0755813  0.442461  2.23175   0.0       0.0
 0.0401385  0.234975  0.654138  2.36563   0.0
 0.0177604  0.103971  0.289442  0.604262  2.07777

# Compute OGM H Matrix

In [ ]:
function compute_theta(N)
    θ = zeros(N+1)
    θ[1] = 1.0  # θ₀
    for i in 2:N
        θ[i] = (1 + sqrt(1 + 4*θ[i-1]^2)) / 2
    end
    θ[N+1] = (1 + sqrt(1 + 8*θ[N]^2)) / 2  # θ_N
    return θ
end
function compute_H(N)
    θ = compute_theta(N)
    H = zeros(N, N)
    for i in 1:N
        for k in 1:i-1
            sum_hjk = sum(H[j, k] for j in k:i)
            H[i, k] = (1 / θ[i+1]) * (2 * θ[k] - sum_hjk)
        end
        H[i, i] = 1+ (2*θ[i]-1) / θ[i+1]
    end
    return H
end

compute_H (generic function with 1 method)

In [ ]:
compute_H(3)

3×3 Matrix{Float64}:
 1.61803    0.0       0.0
 0.174133   2.01939   0.0
 0.0570632  0.334054  1.92996

In [ ]:
compute_H(4)

4×4 Matrix{Float64}:
 1.61803    0.0       0.0       0.0
 0.174133   2.01939   0.0       0.0
 0.0755813  0.442461  2.23175   0.0
 0.0299157  0.17513   0.487537  2.01782

In [ ]:
compute_H(5)

5×5 Matrix{Float64}:
 1.61803    0.0       0.0       0.0       0.0
 0.174133   2.01939   0.0       0.0       0.0
 0.0755813  0.442461  2.23175   0.0       0.0
 0.0401385  0.234975  0.654138  2.36563   0.0
 0.0177604  0.103971  0.289442  0.604262  2.07777